In [2]:
#---import libraries relating to data manipulation, visualization---------------------------------------------------
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option("max_colwidth", None)

import numpy as np
import json

#-------import libraries to evaluate the deep learning model---------------------------------------------------------------
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

#------ import Deep Learning Libraries-------#

import torch
import torch.nn as nn
import torch.optim as optim


import optuna
from optuna.trial import Trial
import pickle


import sys
sys.path.append('../../../src')  # Add the 'src' directory to the path

#### Import Data Loading and Processing Functions<
from multimodal.helper_functions import reproducible_comp, get_final_df
from multimodal.model_architecture import MultimodalModel

reproducible_comp()

In [3]:
file_path = "../../../data/processed_data/force_inner.pkl"
with open(file_path, 'rb') as picklefile:
    data = pickle.load(picklefile)

features_force_inner, category_force_inner, category_names, df_force_inner = get_final_df(data, is_acc_data=0, is_multilabel=1)

file_path = "../../../data/processed_data/acc_inner.pkl"
with open(file_path, 'rb') as picklefile:
    data = pickle.load(picklefile)

features_acc_inner, category_acc_inner, category_names, df_acc_inner = get_final_df(data, is_acc_data=1, is_multilabel=1)

In [4]:
file_path = "../../../data/processed_data/force_outer.pkl"
with open(file_path, 'rb') as picklefile:
    data = pickle.load(picklefile)

features_force_outer, category_force_outer, category_names, df_force_outer = get_final_df(data, is_acc_data=0, is_multilabel=1)

file_path = "../../../data/processed_data/acc_outer.pkl"
with open(file_path, 'rb') as picklefile:
    data = pickle.load(picklefile)

features_acc_outer, category_acc_outer, category_names, df_acc_outer = get_final_df(data, is_acc_data=1, is_multilabel=1)

In [5]:
merged_features_force = np.concatenate((features_force_inner,features_force_outer), axis=0)
merged_category_force = np.concatenate((category_force_inner,category_force_outer), axis=0)

merged_features_acc = np.concatenate((features_acc_inner,features_acc_outer), axis=0)
merged_category_acc = np.concatenate((category_acc_inner,category_acc_outer), axis=0)

In [7]:
labels_force = merged_category_force.copy()
features_force = merged_features_force.copy()
print("Force Features Shape: ", features_force.shape)
print("Force Labels Shape: ", labels_force.shape)

labels_acc = merged_category_acc.copy()
features_acc = merged_features_acc.copy()
print("Acceleration Features Shape: ", features_acc.shape)
print("Acceleration Labels Shape: ", labels_acc.shape)

Force Features Shape:  (2680, 1)
Force Labels Shape:  (2680, 4)
Acceleration Features Shape:  (2680, 3)
Acceleration Labels Shape:  (2680, 4)


#### For Force

In [8]:
max_length = 150
for i in range (len(features_force)):
    if(len(features_force[i][0]<max_length)):
        features_force[i][0]=np.pad(features_force[i][0], (0,max_length-len(features_force[i][0])), 'constant', constant_values=(0))
feat_force= np.zeros(len(features_force) *max_length).reshape(-1, max_length)
for i in range(len(features_force)):
    feat_force[i]= np.concatenate((features_force[i][0]), axis=None)
    

print("Updated Features Shape: ", feat_force.shape)

Updated Features Shape:  (2680, 150)


#### For Acceleration

In [9]:
max_length = 500

for i in range (len(features_acc)):
    if(len(features_acc[i][0]<max_length)):
        features_acc[i][0]=np.pad(features_acc[i][0], (0,max_length-len(features_acc[i][0])), 'constant', constant_values=(0))
        
    if(len(features_acc[i][1]<max_length)):
        features_acc[i][1]=np.pad(features_acc[i][1], (0,max_length-len(features_acc[i][1])), 'constant', constant_values=(0))
    
    if(len(features_acc[i][2]<max_length)):
        features_acc[i][2]=np.pad(features_acc[i][2], (0,max_length-len(features_acc[i][2])), 'constant', constant_values=(0))

feat_acc = np.zeros(len(features_acc)*3*max_length).reshape(-1, 3*max_length)
for i in range(len(features_acc)):
    feat_acc[i]= np.concatenate((features_acc[i][0],features_acc[i][1], features_acc[i][2]), axis=None)
    

print("Updated Features_acc Shape: ", feat_acc.shape)

Updated Features_acc Shape:  (2680, 1500)


In [10]:
X_train_force, X_test_force, y_train_force, y_test_force = train_test_split(feat_force, labels_force, test_size=0.1, 
                                                    random_state=2, shuffle=True, stratify=labels_force)

X_train_acc, X_test_acc, y_train_acc, y_test_acc = train_test_split(feat_acc, labels_acc, test_size=0.1, 
                                                    random_state=2, shuffle=True, stratify=labels_acc)                                                   

In [12]:
X_train_force = X_train_force.reshape(-1, 1, feat_force.shape[1])
X_test_force = X_test_force.reshape(-1, 1, feat_force.shape[1])

X_train_acc = X_train_acc.reshape(-1, 1, feat_acc.shape[1])
X_test_acc = X_test_acc.reshape(-1, 1, feat_acc.shape[1])

print(X_train_force.shape)
print(y_train_force.shape)
print(X_test_force.shape)
print(y_test_acc.shape)

print(X_train_acc.shape)
print(y_train_acc.shape)
print(X_test_acc.shape)
print(y_test_acc.shape)

(2412, 1, 150)
(2412, 4)
(268, 1, 150)
(268, 4)
(2412, 1, 1500)
(2412, 4)
(268, 1, 1500)
(268, 4)


In [13]:
from torch.utils.data import DataLoader, TensorDataset

input_channels = 1
num_classes = 4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load your multimodal data and labels for both modalities
x_train_force = torch.tensor(X_train_force, dtype=torch.float32)
y_train_force = torch.tensor(y_train_force, dtype=torch.float32)

x_train_acc = torch.tensor(X_train_acc, dtype=torch.float32)
y_train_acc = torch.tensor(y_train_acc, dtype=torch.float32)

# Define the objective function for hyperparameter optimization
def multimodal_objective(trial: Trial):
    try:
        # Define hyperparameters to search for
        lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)  # Suggest a learning rate in log scale
        dropout_prob = trial.suggest_float("dropout_prob", 0.0, 0.5)
        force_n_layers = trial.suggest_int("force_n_layers", 1, 7)
        force_kernel_size = trial.suggest_int("force_kernel_size", 3, 9)
        force_stride = trial.suggest_int("force_stride", 1, 7)
        force_padding = trial.suggest_int("force_padding", 1, 10)
        force_hidden_channels = trial.suggest_int("force_hidden_channels", 16, 256)
        force_pooling_type = trial.suggest_categorical("force_pooling_type", ["max", "avg"])
        force_use_batch_norm = trial.suggest_categorical("force_use_batch_norm", [True, False])

        acc_n_layers = trial.suggest_int("acc_n_layers", 1, 7)
        acc_kernel_size = trial.suggest_int("acc_kernel_size", 3, 9)
        acc_stride = trial.suggest_int("acc_stride", 1, 7)
        acc_padding = trial.suggest_int("acc_padding", 1, 10)
        acc_hidden_channels = trial.suggest_int("acc_hidden_channels", 16, 256)
        acc_pooling_type = trial.suggest_categorical("acc_pooling_type", ["max", "avg"])
        acc_use_batch_norm = trial.suggest_categorical("acc_use_batch_norm", [True, False])


        # Split the data into training and validation sets
        x_train_split_force, x_valid_split_force, y_train_split_force, y_valid_split_force = train_test_split(
            x_train_force, y_train_force, test_size=0.2, random_state=42
        )
        x_train_split_acc, x_valid_split_acc, y_train_split_acc, y_valid_split_acc = train_test_split(
            x_train_acc, y_train_acc, test_size=0.2, random_state=42
        )

        # Create datasets and data loaders for both modalities
        train_dataset_force = TensorDataset(x_train_split_force, y_train_split_force)
        valid_dataset_force = TensorDataset(x_valid_split_force, y_valid_split_force)

        train_dataset_acc = TensorDataset(x_train_split_acc, y_train_split_acc)
        valid_dataset_acc = TensorDataset(x_valid_split_acc, y_valid_split_acc)

        train_loader_force = DataLoader(train_dataset_force, batch_size=64, shuffle=True)
        valid_loader_force = DataLoader(valid_dataset_force, batch_size=64, shuffle=False)

        train_loader_acc = DataLoader(train_dataset_acc, batch_size=64, shuffle=True)
        valid_loader_acc = DataLoader(valid_dataset_acc, batch_size=64, shuffle=False)

        # Create the multimodal model with the suggested hyperparameters
        model = MultimodalModel(input_channels, num_classes, 
                 force_n_layers, force_kernel_size, force_stride, force_padding, force_hidden_channels,
                 force_pooling_type, force_use_batch_norm,
                 acc_n_layers, acc_kernel_size, acc_stride, acc_padding, acc_hidden_channels,
                 acc_pooling_type, acc_use_batch_norm, dropout_prob).to(device)

        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)

        num_epochs = 50

        model.train()
        for epoch in range(num_epochs):
            # Training loop
            running_loss = 0.0
            for (inputs_acc, labels), (inputs_force,lab) in zip(train_loader_acc, train_loader_force):
                inputs_acc, labels, inputs_force = inputs_acc.to(device), labels.to(device), inputs_force.to(device)

                optimizer.zero_grad()

                outputs = model(inputs_force, inputs_acc)
                loss = criterion(outputs, labels.float())
                loss.backward()
                optimizer.step()

                running_loss += loss.item()

            train_loss = running_loss / len(train_loader_acc)

            # Validation loss calculation (similar to training loop)
            model.eval()
            valid_loss = 0.0
            with torch.no_grad():
                for (inputs_acc_valid, labels_valid), (inputs_force_valid,lab) in zip(valid_loader_acc, valid_loader_force):
                    inputs_acc_valid, labels_valid, inputs_force_valid = inputs_acc_valid.to(device), labels_valid.to(device), inputs_force_valid.to(device)
                    outputs_valid = model(inputs_force_valid, inputs_acc_valid)
                    loss_valid = criterion(outputs_valid, labels_valid.float())
                    valid_loss += loss_valid.item()

            valid_loss = valid_loss / len(valid_loader_acc)
            if (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {train_loss}, Validation Loss: {valid_loss}")

        # Calculate and return the final validation loss
        return valid_loss
    
    except Exception as e:
        # Print the error for debugging purposes (optional)
        print(f"Trial failed with error: {e}")

        # Return a high validation loss to indicate failure
        return 1.0  # High validation loss


In [15]:
# Create an Optuna study for multimodal model hyperparameter optimization
study = optuna.create_study(
    study_name="Case#9: Multi Modal Merged",
    direction="minimize",  # Minimize final validation loss
    storage="sqlite:///../../database_sqllite//db.sqlite3",
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
)

# Define the number of trials (hyperparameter combinations) you want to search
num_trials = 150

# Perform hyperparameter optimization for the multimodal model
study.optimize(multimodal_objective, n_trials=num_trials, n_jobs=1)


[I 2023-10-11 00:31:36,811] A new study created in RDB with name: Case#9: Multi Modal Merged


Epoch 10/50, Training Loss: 0.239427603540882, Validation Loss: 0.3219570629298687
Epoch 20/50, Training Loss: 0.12500581457730262, Validation Loss: 0.37958819419145584
Epoch 30/50, Training Loss: 0.020142037301294265, Validation Loss: 0.6773548759520054
Epoch 40/50, Training Loss: 0.0009394267220951376, Validation Loss: 0.966538555920124


[I 2023-10-11 00:32:58,450] Trial 0 finished with value: 1.0952163711190224 and parameters: {'lr': 0.001782241196279316, 'dropout_prob': 0.14269982125781172, 'force_n_layers': 7, 'force_kernel_size': 6, 'force_stride': 5, 'force_padding': 5, 'force_hidden_channels': 25, 'force_pooling_type': 'max', 'force_use_batch_norm': False, 'acc_n_layers': 4, 'acc_kernel_size': 3, 'acc_stride': 2, 'acc_padding': 4, 'acc_hidden_channels': 89, 'acc_pooling_type': 'avg', 'acc_use_batch_norm': True}. Best is trial 0 with value: 1.0952163711190224.


Epoch 50/50, Training Loss: 0.0003668555122379574, Validation Loss: 1.0952163711190224
Epoch 10/50, Training Loss: 0.6138200971388048, Validation Loss: 0.6119955778121948
Epoch 20/50, Training Loss: 0.5136406450502334, Validation Loss: 0.515052005648613
Epoch 30/50, Training Loss: 0.4810181615814086, Validation Loss: 0.4768110141158104
Epoch 40/50, Training Loss: 0.4631433756120743, Validation Loss: 0.46654699742794037


[I 2023-10-11 02:01:56,048] Trial 1 finished with value: 0.44984790682792664 and parameters: {'lr': 1.7146860262856232e-05, 'dropout_prob': 0.23007031842583364, 'force_n_layers': 7, 'force_kernel_size': 7, 'force_stride': 2, 'force_padding': 5, 'force_hidden_channels': 208, 'force_pooling_type': 'max', 'force_use_batch_norm': True, 'acc_n_layers': 4, 'acc_kernel_size': 8, 'acc_stride': 2, 'acc_padding': 8, 'acc_hidden_channels': 93, 'acc_pooling_type': 'max', 'acc_use_batch_norm': False}. Best is trial 1 with value: 0.44984790682792664.


...

Epoch 50/50, Training Loss: 0.5037274427952305, Validation Loss: 0.504856288433075
Epoch 10/50, Training Loss: 0.45376274854906146, Validation Loss: 0.45148326829075813
Epoch 20/50, Training Loss: 0.42019278484006084, Validation Loss: 0.4143783561885357
Epoch 30/50, Training Loss: 0.3942677897791709, Validation Loss: 0.38149646297097206
Epoch 40/50, Training Loss: 0.35231862721904633, Validation Loss: 0.3624439090490341


[I 2023-10-11 05:20:31,596] Trial 148 finished with value: 0.3259611763060093 and parameters: {'lr': 0.0008496016774900051, 'dropout_prob': 0.2844720111379995, 'force_n_layers': 6, 'force_kernel_size': 8, 'force_stride': 4, 'force_padding': 6, 'force_hidden_channels': 204, 'force_pooling_type': 'avg', 'force_use_batch_norm': True, 'acc_n_layers': 4, 'acc_kernel_size': 5, 'acc_stride': 3, 'acc_padding': 6, 'acc_hidden_channels': 35, 'acc_pooling_type': 'avg', 'acc_use_batch_norm': False}. Best is trial 138 with value: 0.2858945671468973.


Epoch 50/50, Training Loss: 0.32552905044248026, Validation Loss: 0.3259611763060093
Epoch 10/50, Training Loss: 0.49089516362836283, Validation Loss: 0.494764294475317
Epoch 20/50, Training Loss: 0.4724936802541056, Validation Loss: 0.46233468130230904
Epoch 30/50, Training Loss: 0.4474673242338242, Validation Loss: 0.44659991934895515
Epoch 40/50, Training Loss: 0.4033730549197043, Validation Loss: 0.41020066663622856


[I 2023-10-11 05:21:43,743] Trial 149 finished with value: 0.3782809339463711 and parameters: {'lr': 0.0005535015415739375, 'dropout_prob': 0.2677504364229746, 'force_n_layers': 6, 'force_kernel_size': 8, 'force_stride': 5, 'force_padding': 6, 'force_hidden_channels': 219, 'force_pooling_type': 'avg', 'force_use_batch_norm': True, 'acc_n_layers': 7, 'acc_kernel_size': 6, 'acc_stride': 3, 'acc_padding': 7, 'acc_hidden_channels': 28, 'acc_pooling_type': 'avg', 'acc_use_batch_norm': False}. Best is trial 138 with value: 0.2858945671468973.


Epoch 50/50, Training Loss: 0.3773023034295728, Validation Loss: 0.3782809339463711


In [16]:
# Print the best hyperparameters and their associated final validation loss
print("Best hyperparameters:")
print(study.best_params)
print("Best final validation loss:")
print(study.best_value)

Best hyperparameters:
{'lr': 0.0009110549600440013, 'dropout_prob': 0.26644130402468813, 'force_n_layers': 6, 'force_kernel_size': 8, 'force_stride': 4, 'force_padding': 6, 'force_hidden_channels': 225, 'force_pooling_type': 'avg', 'force_use_batch_norm': True, 'acc_n_layers': 4, 'acc_kernel_size': 6, 'acc_stride': 3, 'acc_padding': 7, 'acc_hidden_channels': 74, 'acc_pooling_type': 'avg', 'acc_use_batch_norm': False}
Best final validation loss:
0.2858945671468973


In [17]:
# Specify the path where you want to save the JSON file
config_file_path = "../best_configs/multimodal_merged_best_configs.json"

# Save the best hyperparameters and validation loss to a JSON file
best_configs = {
    "best_params": study.best_params,
    "best_value": study.best_value,
}

with open(config_file_path, "w") as config_file:
    json.dump(best_configs, config_file)

In [17]:
# Load the saved configuration
with open(config_file_path, "r") as config_file:
    saved_configs = json.load(config_file)

# Compare the new best value with the saved one
if study.best_value < saved_configs["best_value"]:
    print("New best value found. Replacing saved configuration.")
    saved_configs["best_params"] = study.best_params
    saved_configs["best_value"] = study.best_value

    # Update the saved configuration file
    with open(config_file_path, "w") as config_file:
        json.dump(saved_configs, config_file)

In [16]:
# Save the best hyperparameters and validation loss to a JSON file
best_configs = {
    "best_params": study.best_params,
    "best_value": study.best_value,
}

with open(config_file_path, "w") as config_file:
    json.dump(best_configs, config_file)

# Thank You